In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn import metrics
import numpy as np
from sklearn.impute import SimpleImputer

In [2]:
#Loading training data
test_data = pd.read_csv('./Resources/test_modified.csv')
train_data = pd.read_csv('./Resources/train_modified.csv')

# train_data.info()
train_data["Item_Outlet_Sales"]

0       3735.1380
1        443.4228
2       2097.2700
3        732.3800
4        994.7052
          ...    
8518    2778.3834
8519     549.2850
8520    1193.1136
8521    1845.5976
8522     765.6700
Name: Item_Outlet_Sales, Length: 8523, dtype: float64

In [11]:
sales_predict_df = pd.concat([train_data, test_data], ignore_index=True)
sales_predict_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 33 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Item_Identifier         14204 non-null  object 
 1   Item_Weight             14204 non-null  float64
 2   Item_Visibility         14204 non-null  float64
 3   Item_MRP                14204 non-null  float64
 4   Outlet_Identifier       14204 non-null  object 
 5   Item_Outlet_Sales       8523 non-null   float64
 6   Outlet_Years            14204 non-null  int64  
 7   Item_Fat_Content_0      14204 non-null  int64  
 8   Item_Fat_Content_1      14204 non-null  int64  
 9   Outlet_Location_Type_0  14204 non-null  int64  
 10  Outlet_Location_Type_1  14204 non-null  int64  
 11  Outlet_Location_Type_2  14204 non-null  int64  
 12  Outlet_Size_0           14204 non-null  int64  
 13  Outlet_Size_1           14204 non-null  int64  
 14  Outlet_Size_2           14204 non-null

In [4]:
#Create features and target array
features_df = sales_predict_df.drop(columns= ['Item_Identifier', 'Outlet_Identifier', 'Item_Outlet_Sales'])
target_df = sales_predict_df['Item_Outlet_Sales']
#Set the target array values
X = features_df.values
y = target_df.values


In [5]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [6]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:
print("Any missing in X_train_scaled:", np.isnan(X_train_scaled).any())
print("Any missing in y_train:", np.isnan(y_train).any())


Any missing in X_train_scaled: False
Any missing in y_train: True


In [16]:

mask = ~np.isnan(y_train)
X_train_scaled = X_train_scaled[mask]
y_train = y_train[mask]


In [18]:

imputer = SimpleImputer(strategy="mean")
y_train = imputer.fit_transform(y_train.reshape(-1, 1)).ravel()


In [22]:
# Uee XGBoost to define regressor model
regressor = XGBRegressor()

# Fit the model to the training data
regressor.fit(X_train_scaled, y_train)



XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [23]:
#Running the model on the training data to predict sales
sales_data_predictions = regressor.predict(X_test_scaled)


In [25]:
# Check and remove NaNs in y_test
mask_test = ~np.isnan(y_test)
X_test_scaled = X_test_scaled[mask_test]
y_test = y_test[mask_test]

# Re-run the prediction on modified X_test_scaled
sales_data_predictions = regressor.predict(X_test_scaled)

# Calculate the R-squared value again
r2_sales = metrics.r2_score(y_test, sales_data_predictions)
print('R Squared value = ', r2_sales)


R Squared value =  0.5352553307473309
